# LLM from scratch Series

Stage 1: Data Preperation and sampling, Attention Mechanism, LLM architecture

BUILDING AN LLM

Stage 2: Training Loop, Model Evaluation, Loading Pretrained Weights

FOUNDATION Model

Stage 3: Finetuning 

Classifier OR Personal Assistant 

## Satge 1: Data Prep & Sampling-Tokenization
How do you prepare inpur text for training LLMs?

Step 1: Splitting text into individual word and subword token

Step 2: Converting tokens in token IDs

step 3: Encode token IDs into vector representations


## GPT Tokenization Process

<img src="tokenization.png" alt="GPT Tokenization Flow" width="800"/>


# Tokenization

## Step 1: Cretaing tokens

In [107]:
with open("the-verdict.txt", "r", encoding ="utf-8") as f:
    raw_text= f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [80]:
import re
text= "Hello, world. This, is a text"
result= re.split(r'(\s)',text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'text']


In [81]:
import re
text= "Hello, world. This, is a text"
result= re.split(r'([,.]|\s)',text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'text']


In [82]:
result= [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'text']


In [83]:
#simple tokenization scheme
import re
text= "Hello, world. This, is-- a text?"
result=re.split(r'([,.:;?_!"()\']|--|\s)', text)

result= [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', '--', 'a', 'text', '?']


In [84]:

result= re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
result= [item.strip() for item in result if item.strip()]
print(len(result))

4690


In [85]:
result[:30]

['I',
 'HAD',
 'always',
 'thought',
 'Jack',
 'Gisburn',
 'rather',
 'a',
 'cheap',
 'genius',
 '--',
 'though',
 'a',
 'good',
 'fellow',
 'enough',
 '--',
 'so',
 'it',
 'was',
 'no',
 'great',
 'surprise',
 'to',
 'me',
 'to',
 'hear',
 'that',
 ',',
 'in']

## Step 2: Creating Token ids

Creating Vocabulary in alphabetical form of tokens and assigning unique to each unique token


In [86]:
all_words= sorted(set(result))
print(len(all_words))

1130


In [87]:
vocab={token:integers for integers, token in enumerate(all_words)}
#later when we get an ouput from LLM it eill be in numbers so we need inverse version of vocabulary
# that converts toke IDs to back to corresponding text tokens

### Let's make a tokinizer class in Python that have encode(text token to token ids) method and decode method(tokens is to text token)

In [88]:
class SimpleTokenizer:
    def __init__(self,vocab):
        self.str_to_int= vocab
        self.int_to_str= {id:token for token, id in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        #removing the unnecessary space
        preprocessed= [item.strip() for item in preprocessed if item.strip()]
        ids= [self.str_to_int[s] for s in preprocessed]
        return ids


    def decode(self,id):
        text=" ".join([self.int_to_str[i] for i in id])
        #replace spaces before the specified punctuations
        text= re.sub(r'\s+([,.:;?_!"()\'])',r'\1',text)
        return text

In [89]:
tokens= SimpleTokenizer(vocab=vocab)

In [90]:
id=tokens.encode("This is ")

In [91]:
tokens.decode(id)

'This is'

## ADDING SPECIAL CONTEXT TOKENS
We will modify the python class to handle unknown tokens

### ADDING SPECIAL CONTEXT TOKENS

In the previous section, we implemented a simple tokenizer and applied it to a passage
from the training set. 

In this section, we will modify this tokenizer to handle unknown
words.


In particular, we will modify the vocabulary and tokenizer we implemented in the
previous section, SimpleTokenizerV2, to support two new tokens, <|unk|> and
<|endoftext|>

<div class="alert alert-block alert-warning">

We can modify the tokenizer to use an <|unk|> token if it
encounters a word that is not part of the vocabulary. 

Furthermore, we add a token between
unrelated texts. 

For example, when training GPT-like LLMs on multiple independent
documents or books, it is common to insert a token before each document or book that
follows a previous text source

</div>



In [92]:
#adding the end of text and unkown tokens in the Vocabulary
all_tokens= sorted(list(set(result)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])
vocab={tokens: integer for integer,tokens in enumerate(all_tokens)}

In [93]:
len(vocab)

1132

In [94]:
for i,item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


#### New Tokenizer class to handle new tokens

In [95]:
class SimpleTokenizerV2:
    def __init__(self,vocab):
        self.str_to_int= vocab
        self.int_to_str= {id:token for token, id in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        #removing the unnecessary space
        preprocessed= [item.strip() for item in preprocessed if item.strip()]
        preprocessed= [item if item in self.str_to_int else '<|unk|>' for item in preprocessed]
        ids= [self.str_to_int[s] for s in preprocessed]
        return ids


    def decode(self,id):
        text=" ".join([self.int_to_str[i] for i in id])
        #replace spaces before the specified punctuations
        text= re.sub(r'\s+([,.:;?_!"()\'])',r'\1',text)
        return text

In [96]:
tokenizer= SimpleTokenizerV2(vocab)
text1="Hello, do you like tea?" 
text2="In the sunlit terraces of the places."
text= " <|endoftext|> ".join((text1,text2))

In [97]:
text

'Hello, do you like tea? <|endoftext|> In the sunlit terraces of the places.'

In [98]:
tokenizer.encode(text=text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [99]:
tokenizer.decode(tokenizer.encode(text=text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

### Tokenization algorithm
- Word based
- Sub-word based
- Chaarcter based

## Byte Pair encoding- Sub word encoding

- Rule 1: Do not split frequently used words into smaller subwords
- Rule 2: Split the rare words into smaller, meaningful subwords

Example: "boy" should not be split, "boys" should be split into "boy" and "s"

In [101]:
import importlib
import tiktoken
#print('tiktoken version:', importlib.metadata.version("tiktoken"))

In [102]:
tokenizer= tiktoken.get_encoding("gpt2")

In [103]:
text= (" Hello, do you like tea? <|endoftext|> In the sunlit terraces" "of someunknownPlace")
integers= tokenizer.encode(text,allowed_special={"<|endoftext|>"})
print(integers)

[18435, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271]


In [104]:
tokenizer.decode(integers)

' Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace'

## Create Input-Target pairs
- The last step before we create vector embeddings is to create input-target pairs

In [105]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
    raw_text= f.read()

enc_text= tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [106]:
context_window=4
x= enc_sample[:context_window]
y=enc_sample[1:context_window+1]
print(f"x: {x}")
print(f"y:      {y}")

NameError: name 'enc_sample' is not defined

In [108]:
for i in range(1,context_window+1):
    context= enc_sample[:i]
    desired= enc_sample[i]
    print(context,"--->",desired)

NameError: name 'enc_sample' is not defined

In [ ]:
for i in range(1,context_window+1):
    context= enc_sample[:i]
    desired= enc_sample[i]
    print(tokenizer.decode(context),"--->",tokenizer.decode([desired]))

 and --->  established
 and established --->  himself
 and established himself --->  in
 and established himself in --->  a


## IMPLEMENTING A DATA LOADER
- Our goal is to create a data loader that creates two tensors a input tensor that the LLM sees and a target tensor that includes the tagets for the LLMs to predict

#### Building A Dataset Class


In [109]:
#Building A Dataset Class

from torch.utils.data import Dataset,DataLoader
import torch
class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length,stride):

        self.input_ids=[]
        self.target_ids=[]

        #tokenize the entire text
        tokens_id= tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        for i in range(0, len(tokens_id)- max_length,stride):
            input_chunk=tokens_id[i:i+max_length]
            target_chunk=tokens_id[i+1: i+1+max_length+1]

            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    



In [110]:
def create_dataloaderv1(text,batch_size=4, max_length=256,
                        stride=128,shuffle=True,drop_last=True,
                        num_worker=0):
    tokenizer= tiktoken.get_encoding("gpt2")

    #Create the dataset
    dataset= GPTDatasetV1(text,tokenizer,max_length,stride)

    #Create the dataloader
    dataloader= DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_worker
    )
    return dataloader

In [111]:
data=create_dataloaderv1(raw_text,batch_size=2, max_length=6,
                        stride=128,shuffle=True,drop_last=True,
                        num_worker=0)

In [112]:
for i, n in data:
    print(i,n)


tensor([[ 1021,   757,   438, 10919,   257,   410],
        [10197,   832,   262, 46475,   286, 18113]]) tensor([[  757,   438, 10919,   257,   410,  5040,   329],
        [  832,   262, 46475,   286, 18113,   544,   338]])
tensor([[  262,  1633,   286, 24380,   329, 20728],
        [ 2612,  4369,    11,   523,   326,   612]]) tensor([[ 1633,   286, 24380,   329, 20728,   287,   257],
        [ 4369,    11,   523,   326,   612,   550,   587]])
tensor([[  503,  4291,   262,  4252, 18250,  8812],
        [19672,   484,   550,   262,  1295,   286]]) tensor([[ 4291,   262,  4252, 18250,  8812,   558,    13],
        [  484,   550,   262,  1295,   286, 15393,   438]])
tensor([[ 1310,  1165,   881, 40642,   972,  6654],
        [  383,  8631,  3872,   373,    11,   314]]) tensor([[ 1165,   881, 40642,   972,  6654,   832,   616],
        [ 8631,  3872,   373,    11,   314,  1422,   470]])
tensor([[ 6164,    25,   366, 16773,   290,   766],
        [  616,   705, 23873,  2350,     6, 14707]])

# Token Embeddings/Vector Embeddings

## Core Concepts

### Token Embeddings
- Dense vector representations that map discrete tokens into continuous vector space
- Each token (word/subword) becomes a fixed-length vector of floating-point numbers
- Dimension typically ranges from 256 to 1024 based on model size

### Why Embeddings Matter
- Transform symbolic text into numerical format for neural networks
- Capture semantic relationships between tokens
- Enable mathematical operations on language
- Foundation for contextual representations

## Architecture Components

### 1. Vocabulary System
- Token dictionary mapping words to unique IDs
- Handles unknown tokens (UNK)
- Special tokens (PAD, BOS, EOS)
- Vocabulary size impacts model complexity

### 2. Embedding Layer
- Learned weight matrix mapping token IDs to vectors
- Initialized randomly, refined during training
- Shared across model's encoder/decoder components
- Enables parameter sharing and efficient learning

### 3. Positional Encoding
- Adds position information to token embeddings
- Can be learned or fixed sinusoidal encodings
- Essential for capturing token order in sequences

## Training Process

### 1. Initialization
- Random initialization of embedding weights
- Xavier/Glorot or other suitable initialization methods
- Embedding dimension chosen based on vocabulary size

### 2. Learning
- Updated via backpropagation
- Learns from contextual predictions
- Optimized to capture semantic relationships

### 3. Output
- Final embeddings reflect learned language patterns
- Similar words cluster in vector space
- Enables meaningful vector arithmetic

## Key Considerations

### Dimensionality
- Higher dimensions capture more nuanced relationships
- Trade-off between expressiveness and computation
- Must balance with model size constraints

### Context Window
- Size of context affects learning quality
- Longer contexts capture broader relationships
- Impacts computational requirements

### Normalization
- L2 normalization often applied
- Helps stabilize training
- Prevents embedding magnitude issues

This overview covers the foundational concepts for implementing embeddings in language models from scratch. Each component builds upon the previous, creating a complete system for neural language understanding.

## Test with Token embeddings

### Import trained word2vec model

In [113]:
import gensim.downloader as api
model= api.load("word2vec-google-news-300")

In [114]:
word_vector=model
print(word_vector['laptop'])
print(word_vector['laptop'].shape)

[ 2.64892578e-02 -1.64062500e-01 -7.01904297e-03  2.79296875e-01
  8.88671875e-02  1.89453125e-01 -2.29492188e-02  1.32812500e-01
  4.08203125e-01 -2.70996094e-02 -1.28906250e-01 -9.17968750e-02
  4.10156250e-02 -1.75781250e-02  1.21582031e-01  1.49414062e-01
  3.97949219e-02  4.41894531e-02 -3.63769531e-02 -1.58203125e-01
  2.72216797e-02 -1.97753906e-02 -3.51562500e-02 -2.62451172e-02
 -3.73535156e-02  3.61328125e-02 -8.05664062e-02  2.51953125e-01
  2.69531250e-01  5.43594360e-05 -3.19824219e-02  1.43432617e-02
 -2.31445312e-01  1.52343750e-01 -2.83203125e-01 -4.39453125e-01
 -3.06640625e-01  2.70996094e-02 -1.63085938e-01 -1.03515625e-01
 -2.17773438e-01  3.20312500e-01  5.05371094e-02  1.81640625e-01
  1.62109375e-01 -1.55639648e-02 -3.97949219e-02  1.91406250e-01
  9.47265625e-02 -2.08984375e-01 -2.06054688e-01 -3.27148438e-02
  1.21093750e-01 -6.68334961e-03 -2.96020508e-03  9.08203125e-02
  1.44195557e-03  2.09960938e-02  2.89306641e-02  1.75781250e-01
  1.63085938e-01  1.30859

## Similar words

### King + Woman - Man

In [ ]:
#example of using most_similar
print(word_vector.most_similar(positive=['king','women'], negative=['man'],topn=10))

[('queen', 0.4827326238155365), ('queens', 0.466781347990036), ('kumaris', 0.4653734564781189), ('kings', 0.4558638632297516), ('womens', 0.422832190990448), ('princes', 0.4176960587501526), ('Al_Anqari', 0.41725507378578186), ('concubines', 0.40110787749290466), ('monarch', 0.39624831080436707), ('monarchy', 0.39430150389671326)]


### Let us check the similarity b/w a few pair of words

In [ ]:
# Example of calculating similarity
print(word_vector.similarity('women','man'))
print(word_vector.similarity('king','queen'))
print(word_vector.similarity('icecream','diabetes'))
print(word_vector.similarity('sugar','honey'))
print(word_vector.similarity('uncle','nephew'))


0.2883053
0.6510956
0.03228098
0.5061663
0.84866095


### Most similar words

In [ ]:
print(word_vector.most_similar('tower',topn=5))

[('towers', 0.8531749844551086), ('skyscraper', 0.6417425870895386), ('Tower', 0.639177143573761), ('spire', 0.5946877598762512), ('responded_Understood_Atlasjet', 0.5931612849235535)]


## How are token embeddings are created for large language models

### Creating Token Embeddings

In [115]:
import torch
inputs_ids= torch.tensor([2,3,5,1])

In [116]:

vocab_size= 6
output_dim=3
torch.manual_seed(123)

embedding_layer= torch.nn.Embedding(vocab_size,output_dim)

In [117]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


### Getting vector of a id

In [118]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [119]:
print(embedding_layer(inputs_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


# POSITIONAL EMBEDDINGS

# Positional Embeddings

## What They Are
Numerical representations of token positions that get added to token embeddings before processing through transformer layers. Essential since transformers process all tokens simultaneously.

## Types

### Absolute 
- Fixed numbers for each position
- Added directly to token embeddings
- Example: word "cat" at position 3 = token_embedding("cat") + position_embedding(3)

### Relative
- Encodes relationships between positions
- More flexible than absolute
- Added during attention computation

## Implementation
1. Convert tokens to embeddings
2. Generate position vectors
3. Add position vectors to token embeddings
4. Feed combined embeddings to transformer

## Key Point
Both types enable LLMs to understand token order and relationships, while preserving the semantic meaning encoded in the original token embeddings.

## Choice Depends On
- Task requirements
- Sequence length needs
- Data characteristics

In [128]:
vocab_size= 50257 #GPT vocab size using byte pair encoding
output_dim= 256

#embedding layer matrix (lookup matrix)
embedding_layer= torch.nn.Embedding(vocab_size,output_dim)
embedding_layer

Embedding(50257, 256)

In [121]:
max_length=4
datalaoder= create_dataloaderv1(raw_text,batch_size=8,max_length=max_length
                                ,stride=max_length,shuffle=False)

data_iter= iter(datalaoder)
inputs,targets= next(data_iter)

In [122]:
print("Token ID:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token ID:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Inputs shape:
 torch.Size([8, 4])


### Create another embedding layer for Positional Encoding 

In [123]:
context_legth= max_length
pos_embedding_layer= torch.nn.Embedding(context_legth,output_dim)

In [131]:
pos_embeddings= pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings)

tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<EmbeddingBackward0>)


tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

In [2]:
input_embeddings=embedding_layer(torch.tensor(inputs)).shape

NameError: name 'embedding_layer' is not defined

In [1]:
input_embeddings+pos_embeddings

NameError: name 'input_embeddings' is not defined

# Attention Mechanism
- SImplified Self Attention
- Self Attention
- Causal Attention
- Multihead Attention

## IMPLEMENTING A SIMPLIFIED ATTENTION MECHANISM 

In [ ]:
import torch 

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)